In [1]:
import os
import warnings
warnings.filterwarnings("ignore")

In [32]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

### Loading the Model and Testing it

In [4]:
from langchain_core.messages import HumanMessage, SystemMessage

In [5]:
from langchain_google_genai import ChatGoogleGenerativeAI
from google import genai

In [6]:
client = genai.Client(api_key=os.environ["GOOGLE_API_KEY"])

for model in client.models.list():
    print(model.name)

models/embedding-gecko-001
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-flash-latest
models/gemini-1.5-flash
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-2.5-pro-preview-03-25
models/gemini-2.5-flash-preview-05-20
models/gemini-2.5-flash
models/gemini-2.5-flash-lite-preview-06-17
models/gemini-2.5-pro-preview-05-06
models/gemini-2.5-pro-preview-06-05
models/gemini-2.5-pro
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-preview-image-generation
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
models/gemini-2.0-flash-thinking-exp-01-21
models/gemini-2.0-flash-thinking-exp
models/ge

In [7]:
model = ChatGoogleGenerativeAI(model="gemini-2.5-pro", convert_system_message_to_human=True)

In [8]:
message = [SystemMessage(content="You are a helpful assistant."), HumanMessage(content="What is the capital of India?")]

In [9]:
message2 = [SystemMessage(content="You are a helpful assistant and always come up with funny answers."), HumanMessage(content="What is the capital of India?")]

In [10]:
model.invoke(message)

AIMessage(content='The capital of India is **New Delhi**.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--951c54c6-8787-4e12-ac8b-7413e33a2a38-0', usage_metadata={'input_tokens': 14, 'output_tokens': 9, 'total_tokens': 325, 'input_token_details': {'cache_read': 0}})

In [11]:
model.invoke(message2)

AIMessage(content='Ah, a classic! The capital of India is the city that\'s so nice, they put "New" in front of it to distinguish it from its older, wiser, and slightly more chaotic sibling.\n\nIt\'s **New Delhi**!\n\nIt\'s a truly *capital* idea to know that one. I\'ll see myself out.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--e7e9a7a9-65a9-4eee-889e-8e71146d2e69-0', usage_metadata={'input_tokens': 21, 'output_tokens': 73, 'total_tokens': 1093, 'input_token_details': {'cache_read': 0}})

### Use Output Parser

In [12]:
from langchain_core.output_parsers import StrOutputParser

In [13]:
parser = StrOutputParser()

In [14]:
response = model.invoke(message)

In [15]:
response2 = model.invoke(message2)

In [16]:
parser.invoke(response)

'The capital of India is **New Delhi**.'

In [17]:
parser.invoke(response2)

"Oh, that's an easy one! The capital of India is **New Delhi**.\n\nJust be careful not to get it confused with its older sibling, Old Delhi, which is right next door and is probably still complaining about how the music in New Delhi is too loud."

### Chaining with LCEL

In [18]:
chain = model | parser

In [19]:
chain.invoke(message)

'The capital of India is **New Delhi**.'

In [20]:
chain.invoke(message2)

"Why, it's the one and only **New Delhi**!\n\nIt's a city that runs on two things: delicious chai and the sheer audacity of its drivers. The local birds are also rumored to be excellent navigators, having learned from watching the rickshaws."

### Prompt Templating

In [21]:
from langchain_core.prompts import ChatPromptTemplate

In [22]:
prompt_template = ChatPromptTemplate.from_messages(
    [("system", "translate the following text to {language}"), ("user", "{text}")]
)

In [23]:
prompt = prompt_template.invoke({"language": "French", "text": "I started learning a new topic today, I want to be a good person"})

In [24]:
parser.invoke(model.invoke(prompt))

'Of course! Here are a few ways to translate that, from the most direct to slightly more nuanced options.\n\n**Direct translation:**\n\n> **J\'ai commencé à apprendre un nouveau sujet aujourd\'hui, je veux être une bonne personne.**\n\n---\n\nHere are a few other natural-sounding options:\n\n*   **Aujourd\'hui, j\'ai commencé à apprendre un nouveau sujet. Je veux devenir une bonne personne.**\n    *(This version uses "devenir" which means "to become," often used in this context.)*\n\n*   **J\'ai commencé un nouveau sujet aujourd\'hui, et j\'aimerais être une bonne personne.**\n    *(This uses "j\'aimerais," which means "I would like," a slightly softer and more common way to express a wish.)*'

In [25]:
prompt.to_messages()

[SystemMessage(content='translate the following text to French', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I started learning a new topic today, I want to be a good person', additional_kwargs={}, response_metadata={})]

### Chaining the Prompt Template, Model and Parser

In [26]:
chain = prompt_template | model | parser

In [27]:
chain.invoke({"language": "marathi", "text": "I am learning a new language today, I want to be a good person"})

'Of course! Here\'s the translation in Marathi:\n\n**मी आज एक नवीन भाषा शिकत आहे, मला एक चांगला माणूस व्हायचं आहे.**\n\n(Mī āj ek navīn bhāṣā shikat āhe, malā ek cāṅgalā māṇūs vhāyacha āhe.)\n\n---\nHere\'s a breakdown:\n\n*   **मी आज एक नवीन भाषा शिकत आहे** (Mī āj ek navīn bhāṣā shikat āhe) - "I am learning a new language today."\n*   **मला एक चांगला माणूस व्हायचं आहे** (Malā ek cāṅgalā māṇūs vhāyacha āhe) - "I want to be a good person."'

In [28]:
LANGCHAIN_API_KEY=os.getenv("LANGCHAIN_API_KEY")
LANGCHAIN_TRACING=os.getenv("LANGCHAIN_TRACING")

In [29]:
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["LANGCHAIN_TRACING"] = LANGCHAIN_TRACING

### Agents in LangChain

In [31]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

In [33]:
memory = MemorySaver()

In [35]:
search = TavilySearchResults(max_results=2, api_key=os.getenv("TAVILY_API_KEY"))

In [36]:
tools = [search]

In [41]:
agent_executor = create_react_agent(model, tools,checkpointer=memory)

In [42]:
config = {"configurable": {"thread_id": "abc123"}}

In [45]:
for chunk in agent_executor.stream({"messages": [HumanMessage(content="Hi I am Atharva and I live in United States. Can you tell me the current weather in my city?")]}, config):
    print(chunk)
    print("---")

{'agent': {'messages': [AIMessage(content='I can help with that! What city in the United States are you in? ', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--f6f11c64-cc57-4388-8dd8-5ad2c7eb877f-0', usage_metadata={'input_tokens': 98, 'output_tokens': 16, 'total_tokens': 328, 'input_token_details': {'cache_read': 0}})]}}
---


In [46]:
for chunk in agent_executor.stream({"messages": [HumanMessage(content="I am in Florida Orlando")]}, config):
    print(chunk)
    print("---")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': '{"query": "current weather in Orlando, Florida"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--194a430c-6a22-41b0-80dd-0348d949f6f4-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Orlando, Florida'}, 'id': '9d6ceb63-f5af-40db-aa26-1ecae6e775ce', 'type': 'tool_call'}], usage_metadata={'input_tokens': 122, 'output_tokens': 26, 'total_tokens': 272, 'input_token_details': {'cache_read': 0}})]}}
---
{'tools': {'messages': [ToolMessage(content='[{"title": "Orlando weather in August 2025 - Weather25.com", "url": "https://www.weather25.com/north-america/usa/florida/orlando?page=month&month=August", "content": "Patchy rain possible\\nFog\\nOvercast\\nFog\\nMist\\nPartly cloudy\\nLight rain shower\\nLight rain shower [...] w

In [47]:
for chunk in agent_executor.stream({"messages": [HumanMessage(content="Today is 25th August")]}, config):
    print(chunk)
    print("---")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': '{"query": "weather in Orlando, Florida on August 25th"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--d4ca0a79-7713-47a4-902d-8a807d190b7f-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Orlando, Florida on August 25th'}, 'id': 'c9667704-224d-4b93-bed4-4e293b959e7d', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1234, 'output_tokens': 31, 'total_tokens': 1391, 'input_token_details': {'cache_read': 0}})]}}
---
{'tools': {'messages': [ToolMessage(content='[{"title": "Orlando weather in August 2025 - Weather25.com", "url": "https://www.weather25.com/north-america/usa/florida/orlando?page=month&month=August", "content": "1.   Home\\n2.    North America\\n3.    Image 7: weather in United StatesUnited States\\n4.    Flo